# Import and install

In [1]:
!pip install scikit-learn==0.22.2 -q # to solve "AttributeError: 'CRF' object has no attribute 'keep_tempfiles'" when using crfsuite
!pip install sklearn_crfsuite -q
!pip install eli5 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.22.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 requires jinja2<=3.0.0, but you have jinja2 3.1.2 which is inco

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import sys
sys.path.append('/content/drive/My Drive/DTA/Internship/')

In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from process_for_baselines import process_data, test_classifier
from ner_crfsuite import flat_classification_report

# CLASSIFIERS

In [7]:
# The English dataset is in the wrong format so we can't use the process data function from our own module

df = pd.read_csv('drive/MyDrive/DTA/Internship/ner_dataset.csv', encoding = "ISO-8859-1")
df = df[:60000]
df = df.fillna(method='ffill')
X = df.drop(['Tag'], axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()
new_classes = classes.copy()
new_classes.pop()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)

In [8]:
test_classifier(X_train, y_train, X_test, y_test, classes, new_classes, PassiveAggressiveClassifier) 

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=1000, n_iter_no_change=5,
                            n_jobs=None, random_state=42, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)
              precision    recall  f1-score   support

       B-art       0.42      0.25      0.31        20
       B-eve       0.24      0.46      0.32        13
       B-geo       0.60      0.79      0.68       630
       B-gpe       0.87      0.71      0.78       353
       B-nat       0.50      0.14      0.22         7
       B-org       0.61      0.31      0.42       360
       B-per       0.53      0.56      0.54       335
       B-tim       0.83      0.71      0.76       357
       I-art       0.29      0.20      0.24        10
       I-eve       0.62      0.36      0.45       

In [9]:
test_classifier(X_train, y_train, X_test, y_test, classes, new_classes, MultinomialNB, alpha=0.01) 

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
              precision    recall  f1-score   support

       B-art       0.10      0.30      0.15        20
       B-eve       0.24      0.46      0.32        13
       B-geo       0.64      0.57      0.61       630
       B-gpe       0.62      0.78      0.69       353
       B-nat       0.40      0.57      0.47         7
       B-org       0.43      0.44      0.44       360
       B-per       0.45      0.49      0.47       335
       B-tim       0.57      0.71      0.63       357
       I-art       0.23      0.30      0.26        10
       I-eve       0.50      0.57      0.53        14
       I-geo       0.38      0.49      0.43       128
       I-gpe       0.00      0.00      0.00        10
       I-nat       0.00      0.00      0.00         2
       I-org       0.49      0.49      0.49       271
       I-per       0.56      0.50      0.53       372
       I-tim       0.20      0.34      0.25       107

   micro avg       0

In [10]:
test_classifier(X_train, y_train, X_test, y_test, classes, new_classes, Perceptron, n_jobs=-1, max_iter=5)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=5, n_iter_no_change=5, n_jobs=-1,
           penalty=None, random_state=42, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-art       0.29      0.10      0.15        20
       B-eve       0.01      0.54      0.02        13
       B-geo       0.60      0.76      0.67       630
       B-gpe       0.92      0.68      0.79       353
       B-nat       0.00      0.00      0.00         7
       B-org       0.67      0.37      0.48       360
       B-per       0.66      0.44      0.53       335
       B-tim       0.92      0.69      0.79       357
       I-art       0.00      0.00      0.00        10
       I-eve       0.00      0.00      0.00        14
       I-geo       0.77      0.29      0.42       128
       I-gpe       0.00      0.00      0.00        10
       I-nat       0.00      0.00      0.00         2
       I-org       0.72      0.37      0.49       271
       I-per       0.81      0.18      0.30       372
       I-tim       0.40      0.37      0.38       107

   micro avg       0.52      0.50      0.51      2989
   macro avg       0.42   

In [11]:
test_classifier(X_train, y_train, X_test, y_test, classes, new_classes, SGDClassifier)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
              precision    recall  f1-score   support

       B-art       0.26      0.25      0.26        20
       B-eve       0.38      0.46      0.41        13
       B-geo       0.53      0.87      0.66       630
       B-gpe       0.90      0.54      0.68       353
       B-nat       1.00      0.14      0.25         7
       B-org       0.71      0.31      0.43       360
       B-per       0.60      0.51      0.55       335
       B-tim       0.67      0.69      0.68       357
       I-art       0.50      0.10      0.17        10
       I-eve       0.67      0.29    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# CRF SUITE

In [12]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [13]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None


In [14]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

In [15]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [16]:
getter = SentenceGetter(df)
sentences = getter.sentences
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [17]:
crf = sklearn_crfsuite.CRF(
    algorithm='pa',
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='pa', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [21]:
y_pred = crf.predict(X_test)
print(flat_classification_report(y_test, y_pred, labels = new_classes))

              precision    recall  f1-score   support

       B-art       0.40      0.29      0.33        14
       B-eve       0.56      0.29      0.38        17
       B-geo       0.73      0.82      0.77       581
       B-gpe       0.84      0.78      0.81       366
       B-nat       0.00      0.00      0.00         8
       B-org       0.71      0.58      0.63       396
       B-per       0.77      0.84      0.80       304
       B-tim       0.85      0.80      0.82       364
       I-art       0.25      0.09      0.13        11
       I-eve       0.33      0.36      0.35        11
       I-geo       0.64      0.52      0.58       125
       I-gpe       0.00      0.00      0.00         7
       I-nat       0.00      0.00      0.00         4
       I-org       0.78      0.70      0.74       302
       I-per       0.82      0.92      0.87       348
       I-tim       0.71      0.49      0.58       113

   micro avg       0.77      0.74      0.75      2971
   macro avg       0.52   

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-org  -> I-org   1.332414
B-geo  -> I-geo   1.324856
I-org  -> I-org   1.232112
B-tim  -> I-tim   1.098045
B-per  -> I-per   1.064053
I-per  -> I-per   1.020218
I-tim  -> I-tim   1.001252
B-gpe  -> I-gpe   0.985662
O      -> O       0.818177
O      -> B-per   0.783002
I-geo  -> I-geo   0.755419
I-gpe  -> I-gpe   0.688906
O      -> B-org   0.643359
B-art  -> I-art   0.643053
I-art  -> I-art   0.622168
B-eve  -> I-eve   0.606823
B-gpe  -> B-org   0.534133
B-geo  -> B-gpe   0.516391
O      -> B-geo   0.455064
I-eve  -> I-eve   0.434412

Top unlikely transitions:
B-tim  -> B-tim   -0.299693
B-org  -> I-gpe   -0.301804
B-geo  -> B-geo   -0.303232
I-gpe  -> O       -0.319190
B-geo  -> I-org   -0.345359
B-org  -> B-org   -0.361956
I-org  -> I-per   -0.394194
B-geo  -> I-per   -0.410526
B-geo  -> B-org   -0.441790
B-tim  -> B-gpe   -0.453892
O      -> I-per   -0.456106
B-geo  -> B-per   -0.476951
B-gpe  -> B-tim   -0.507680
I-per  -> B-per   -0.509992
B-geo  -> I-gpe  

In [23]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
1.472280 O        bias
1.253417 B-gpe    -1:word.lower():recognize
1.217697 B-geo    -1:word.lower():sponsored
1.168961 B-tim    word[-3:]:day
1.074849 B-org    +1:word.lower():citizenship
1.038050 B-tim    word[-2:]:0s
1.025131 B-gpe    word.lower():bosnia-herzegovina
0.987310 B-org    -1:word.lower():recognize
0.974184 B-org    +1:word.lower():because
0.963110 B-gpe    -1:word.lower():strike
0.962551 B-org    -1:word.lower():observed
0.955110 B-org    +1:word.lower():troops
0.943044 B-gpe    +1:word.lower():pledged
0.910097 I-geo    word.lower():settlement
0.907042 B-geo    +1:word.lower():province
0.906702 B-gpe    word.lower():zambia
0.901639 B-gpe    word.lower():nepal
0.899315 B-org    +1:word.lower():regained
0.897324 B-org    word.lower():al-qaida
0.893489 B-org    +1:word.lower():naval
0.892591 I-gpe    +1:word.lower():returned
0.885403 I-geo    -1:word.lower():bank
0.873758 O        word.lower():jewish
0.866591 B-tim    word[-2:]:ay
0.861658 B-tim    +1:word.low

In [24]:
import eli5
eli5.show_weights(crf, top=10)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,0.818,0.255,-0.106,0.406,-0.111,0.455,-0.665,0.301,-0.168,0.091,0.0,0.643,-0.533,0.783,-0.456,0.403,-0.802
B-art,0.013,0.0,0.643,0.0,0.0,-0.066,-0.041,-0.022,-0.011,0.0,-0.056,-0.058,-0.015,-0.137,-0.07,0.022,0.0
I-art,-0.203,0.0,0.622,0.0,-0.034,0.229,-0.044,-0.029,0.0,0.0,0.0,0.0,0.0,-0.169,-0.05,-0.058,-0.011
B-eve,-0.036,0.0,-0.016,0.0,0.607,-0.04,-0.013,-0.048,-0.086,0.0,0.0,-0.023,-0.06,-0.044,-0.087,-0.045,-0.018
I-eve,0.032,0.0,-0.02,-0.208,0.434,-0.029,-0.025,0.0,0.0,0.0,0.0,-0.011,0.0,-0.043,-0.026,-0.018,0.0
B-geo,0.131,0.267,-0.14,-0.022,-0.115,-0.303,1.325,0.516,-0.511,0.0,-0.042,-0.442,-0.345,-0.477,-0.411,0.387,-0.193
I-geo,0.069,0.0,-0.06,0.0,-0.06,-0.148,0.755,0.0,-0.185,0.0,-0.007,-0.079,-0.265,-0.203,-0.125,0.314,-0.175
B-gpe,0.335,-0.065,-0.063,-0.061,-0.072,-0.269,-0.188,-0.657,0.986,-0.019,0.0,0.534,-0.213,0.174,-0.225,-0.508,-0.096
I-gpe,-0.319,-0.1,0.0,0.0,-0.022,0.159,-0.187,-0.042,0.689,0.0,0.0,-0.049,-0.062,-0.054,-0.023,-0.081,-0.061
B-nat,-0.036,0.0,0.0,0.0,-0.031,-0.09,0.0,-0.015,0.0,-0.031,0.403,0.0,0.0,-0.102,-0.051,0.0,0.0


In [25]:
eli5.show_weights(crf, top=10, targets=['O', 'B-org', 'I-per'])

In [26]:
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

Weight?,Feature
-0.546,word.isupper()
-0.795,word.isdigit()
-0.857,word.istitle()
Weight?,Feature
-0.118,word.isupper()
-0.170,word.istitle()
Weight?,Feature
+0.144,word.isdigit()
+0.067,word.isupper()
+0.057,word.istitle()
